In [1]:
pip install -r requirements.txt.

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'requirements.txt.'


In [2]:
import csv
import os
import pandas as pd
from xml.dom.minidom import parseString
from sklearn.ensemble import (
    BaggingClassifier, ExtraTreesClassifier,
    GradientBoostingClassifier, RandomForestClassifier,
    AdaBoostClassifier
)
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import (
    LogisticRegression, RidgeClassifier,
    SGDClassifier, PassiveAggressiveClassifier
)
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier

In [3]:
def model_list() -> list:
    # List of the models we are running from the sklearn library
    # Using multiple different types, such as regression based,
    # classifiers, and others
    models: list = [
        BaggingClassifier(), RandomForestClassifier(),
        ExtraTreesClassifier(), GaussianProcessClassifier(),
        LogisticRegression(), RidgeClassifier(), SGDClassifier(),
        DecisionTreeClassifier(), ExtraTreeClassifier(),
        PassiveAggressiveClassifier(), KNeighborsClassifier(),
        LinearSVC(), SVC(), GaussianNB(), AdaBoostClassifier(),
        GradientBoostingClassifier()
    ]
    return models

In [4]:
def split(sample: pd.DataFrame):
    # Function to split the dataframes
    # into resulting index (last column)
    # and the data to be used (from column 1-(last-1))

    # Remove the first column (APK name)
    sample = sample.iloc[:, 1:]
    # Get all but last column of given sample
    x_split = sample.iloc[:, :-1]
    # Get last column of the given sample
    y_split = sample.iloc[:, -1]
    return x_split, y_split

In [5]:
def train_models(estimator, samples: tuple) -> dict:
    # Function to run the models and do the predictions
    # All using the sklearn library, the model's
    # respective .fit function & .predict function
    # The confusion matrix & f1 score are taken
    # to give the variables and f1 score
    # Returned in a dictionary to be graphed/used in results
    training: pd.DataFrame = samples[0]
    testing: pd.DataFrame = samples[1]
    x_train, y_train = split(training)
    x_test, y_test = split(testing)
    estimator = estimator.fit(x_train, y_train)
    predicted = estimator.predict(x_test)
    model_info: dict = {
        "confusion": confusion_matrix(y_test, predicted),
        "f1_score": f1_score(
            y_test, predicted, average="weighted"
        ),
        "model": estimator,
        "name": type(estimator).__name__
    }
    return model_info


In [6]:
def print_result(info: dict):
    # Helper function to print the stats/variables for the given model
    print(f"\tStatistics for {info['name']}:")
    print(f"\t\tF1: {info['f1_score']}")
    # If using binary classification (malware/benign ONLY)
    # the confusion matrix gives simple TP/FP/TN/FN below
    # print(f"\t\tTrue Positive (TP): {info['confusion'][0][0]}")
    # print(f"\t\tFalse Positive (FP): {info['confusion'][1][0]}")
    # print(f"\t\tTrue Negative (TN): {info['confusion'][1][1]}")
    # print(f"\t\tFalse Negative (FN): {info['confusion'][0][1]}")

In [7]:
def run_models(samples: tuple) -> tuple:
    # Function to loop through each different model type and
    # run it, keeping track of best one's f1 score
    # This will return the info list of dictionaries,
    # and the top model's name
    top_run: list = [0, ""]
    model_info: list = []
    for estimator in model_list():
        estimator_dict = train_models(estimator, samples)
        if top_run[0] < estimator_dict["f1_score"]:
            top_run[0] = estimator_dict["f1_score"]
            top_run[1] = estimator_dict["name"]
        model_info.append(estimator_dict)
    return model_info, top_run[1]

In [39]:
def create_parameters(feature: str) -> tuple:
    # Split the data (read in using Pandas library)
    # as a dataframe. 80 percent for training,
    # and the other 20 percent for testing the model
    csv_data: pd.DataFrame =  pd.read_csv(r"E:\project sarmin mam\CSV\feature_vectors_syscallsbinders_frequency_5_Cat.csv", on_bad_lines="skip", low_memory=False)
    train_sample: pd.DataFrame = csv_data.sample(frac=0.80)
    test_sample: pd.DataFrame = csv_data.drop(train_sample.index)
    return train_sample, test_sample

In [37]:
 pd.read_csv(r"E:\project sarmin mam\CSV\feature_vectors_syscallsbinders_frequency_5_Cat.csv", on_bad_lines="skip", low_memory=False)

,ACCESS_PERSONAL_INFO___,ALTER_PHONE_STATE___,ANTI_DEBUG_____,CREATE_FOLDER_____,CREATE_PROCESS`_____,CREATE_THREAD_____,DEVICE_ACCESS_____,EXECUTE_____,FS_ACCESS____,FS_ACCESS()____,...,utimes,vfork,vibrate,vibratePattern,wait4,watchRotation,windowGainedFocus,write,writev,Class
0,1,0,0,3,0,14,2,0,3,0,...,0,0,0,0,0,0,0,37,10,1
1,3,0,0,6,0,42,91,0,32,0,...,0,0,0,0,0,0,2,2838,46,1
2,2,0,0,4,0,23,3,0,17,2,...,0,0,0,0,0,0,1,111,20,1
3,1,0,0,4,0,27,9,0,36,0,...,0,0,0,0,0,0,7,987,197,1
4,3,0,0,11,0,18,3,0,16,0,...,0,0,0,0,0,0,1,98,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11593,2,0,0,11,0,61,13,0,66,17,...,0,0,0,0,0,0,10,2764,236,5
11594,6,0,0,10,1,69,50,1,118,15,...,0,0,0,0,1,0,5,4417,295,5
11595,0,0,0,0,0,21,16,0,48,9,...,0,0,0,0,0,0,5,241,67,5
11596,1,0,0,15,0,90,80,0,239,50,...,0,0,0,0,0,0,3,1703,774,5


In [40]:
def predict():
    # Predict the given feature's index. This is the main part of
    # part 3 which runs the sklearn library on a list of models
    # for each feature set. This function loops through the
    # features running each and appending the resulting list of
    # model data as a dictionary with the key as the feature name
    # and value as a list of model information
    final_info: str = ""
    features: list = [
        "Permissions", "Intents", "Receivers", "Activities", "Services"
    ]
    feature_list: dict = {}
    for feature in features:
        final_info += f"Predicting based on feature set: {feature}\n"
        train_sample, test_sample = create_parameters(feature)
        model_info, best_model = run_models((train_sample, test_sample))
        feature_list[f"{feature}"] = model_info
        final_info += f"Best model is {best_model}\n"
    print(final_info)
    print(feature_list)

In [41]:
def extract_permissions(dom) -> list:
    perm_list: list = []
    permissions = dom.getElementsByTagName("uses-permission")
    for perm in permissions:
        perm_list.append(perm.getAttribute("android:name").split(".")[-1])
    return perm_list

In [42]:
def extract_intents(dom) -> list:
    intent_list: list = []
    intents = dom.getElementsByTagName("intent-filter")
    for intent in intents:
        actions = intent.getElementsByTagName("action")
        for action in actions:
            intent_list.append(
                action.getAttribute("android:name").split(".")[-1]
            )
    return intent_list

In [43]:
def extract_receivers(dom) -> list:
    receiver_list: list = []
    receivers = dom.getElementsByTagName("receiver")
    for receiver in receivers:
        receiver_list.append(
            receiver.getAttribute("android:name").split(".")[-1]
        )
    return receiver_list

In [44]:
def extract_activities(dom) -> list:
    activity_list: list = []
    activities = dom.getElementsByTagName("activity")
    for activity in activities:
        activity_list.append(
            activity.getAttribute("android:name").split(".")[-1]
        )
    return activity_list

In [45]:
def extract_services(dom) -> list:
    service_list: list = []
    services = dom.getElementsByTagName("service")
    for service in services:
        service_list.append(
            service.getAttribute("android:name").split(".")[-1]
        )
    return service_list

In [46]:
def feature_array(total: list, values: list) -> list:
    vector = [0]*len(total)
    for val in values:
        vector[total.index(val)] = 1
    return vector

In [47]:
def write_to_csv(info: list, app_name: str, index: str, feature: str):
    # Save the information to a CSV file for sklearn to use
    file_mode: str = "a"
    try:
        int(index)
    except Exception:
        file_mode = "w"
    with open(f"{feature}.csv", file_mode, newline="") as f:
        csvwriter = csv.writer(f)
        csvwriter.writerow([app_name] + info + [index])
        f.close()

In [48]:
def open_features(dir: str) -> tuple:
    # Open the features and create datastructures to
    # use to save the features as a CSV
    manifest_dir: str = f"{dir}_Manifests"
    manifest_files: list = os.listdir(manifest_dir)
    perm_list: dict = {}
    total_perm: set = set()
    intent_list: dict = {}
    total_intent: set = set()
    receiver_list: dict = {}
    total_receiver: set = set()
    activity_list: dict = {}
    total_activity: set = set()
    service_list: dict = {}
    total_service: set = set()
    try:
        manifest_files.remove(".DS_Store")
    except Exception:
        pass
    for app in manifest_files:
        try:
            with open(f"{manifest_dir}/{app}/AndroidManifest.xml", "r") as f:
                data = f.read()
            dom = parseString(data)
            perm_list[app] = extract_permissions(dom)
            total_perm.update(set(perm_list[app]))
            intent_list[app] = extract_intents(dom)
            total_intent.update(set(intent_list[app]))
            receiver_list[app] = extract_receivers(dom)
            total_receiver.update(set(receiver_list[app]))
            activity_list[app] = extract_activities(dom)
            total_activity.update(set(activity_list[app]))
            service_list[app] = extract_services(dom)
            total_service.update(set(service_list[app]))
        except Exception:
            continue
    return (
        perm_list, total_perm, intent_list, total_intent,
        receiver_list, total_receiver, activity_list, total_activity,
        service_list, total_service
    )


In [49]:
def extract_features(directory: list):
    # Extract the APKs features, this uses 5 sets:
    #   Permissions
    #   Intents
    #   Receivers
    #   Activities
    #   Services
    total_perm: set = set()
    permissions: list = []
    total_intents: set = set()
    intents: list = []
    total_receiver: set = set()
    receivers: list = []
    total_activity: set = set()
    activities: list = []
    total_services: set = set()
    services: list = []
    for dir in directory:
        info: tuple = open_features(dir)
        permissions.append(info[0])
        total_perm.update(info[1])
        intents.append(info[2])
        total_intents.update(info[3])
        receivers.append(info[4])
        total_receiver.update(info[5])
        activities.append(info[6])
        total_activity.update(info[7])
        services.append(info[8])
        total_services.update(info[9])
    write_to_csv(list(total_perm), "App Name", "Index", "Permissions")
    write_to_csv(list(total_intents), "App Name", "Index", "Intents")
    write_to_csv(list(total_receiver), "App Name", "Index", "Receivers")
    write_to_csv(list(total_activity), "App Name", "Index", "Activities")
    write_to_csv(list(total_services), "App Name", "Index", "Services")

    for index, permission in enumerate(permissions):
        for app in permission:
            write_to_csv(
                feature_array(list(total_perm), permission[app]), app,
                str(index), "Permissions"
            )
    for index, intent in enumerate(intents):
        for app in intent:
            write_to_csv(
                feature_array(list(total_intents), intent[app]), app,
                str(index), "Intents"
            )
    for index, receiver in enumerate(receivers):
        for app in receiver:
            write_to_csv(
                feature_array(list(total_receiver), receiver[app]), app,
                str(index), "Receivers"
            )
    for index, activity in enumerate(activities):
        for app in activity:
            write_to_csv(
                feature_array(list(total_activity), activity[app]), app,
                str(index), "Activities"
            )
    for index, service in enumerate(services):
        for app in service:
            write_to_csv(
                feature_array(list(total_services), service[app]), app,
                str(index), "Services"
            )

In [50]:
def decode_apps(directory: list):
    # Decode the apps (same as project 1 part 2)
    for dir in directory:
        apk_directory = os.listdir(f"{dir}/")
        if not apk_directory[0].endswith(".apk"):
            for subdir in apk_directory:
                apks = os.listdir(f"{dir}/{subdir}/")
                for app in apks:
                    os.system(
                        f"apktool d {dir}/{subdir}/{app} -o" +
                        f" {dir}_Manifests/{app}"
                    )
        else:
            for app in apk_directory:
                os.system(f"apktool d {dir}/{app} -o {dir}_Manifests/{app}")


In [51]:
def main():
    # Decode the applications (decide which datasets below)
    # default_directories: list = ["benign", "malware"]
    # extra_credit_directories: list = ["Adware", "Ransomware", "Scareware"]
    both_directories: list = [
        "benign", "malware", "Adware", "Ransomware", "Scareware"
    ]
    # decode_apps(both_directories)
    # Extract the features (pass same dataset as above)
    extract_features(both_directories)
    # Predict the models of the CSV's generated above
    predict()

In [53]:
predict()

KeyboardInterrupt: 

In [ ]:
extract_features(both_directories)

In [24]:
import pandas as pd
df=pd.read_csv(r"E:\project sarmin mam\CSV\syscall_unique.csv")
df

,__arm_nr_cacheflush
0,__arm_nr_set_tls
1,_llseek
2,_newselect
3,accept
4,access
...,...
133,utimes
134,vfork
135,wait4
136,write


In [25]:
df=pd.read_csv(r"E:\project sarmin mam\CSV\feature_vectors_syscallsbinders_frequency_5_Cat.csv")
df

,ACCESS_PERSONAL_INFO___,ALTER_PHONE_STATE___,ANTI_DEBUG_____,CREATE_FOLDER_____,CREATE_PROCESS`_____,CREATE_THREAD_____,DEVICE_ACCESS_____,EXECUTE_____,FS_ACCESS____,FS_ACCESS()____,...,utimes,vfork,vibrate,vibratePattern,wait4,watchRotation,windowGainedFocus,write,writev,Class
0,1,0,0,3,0,14,2,0,3,0,...,0,0,0,0,0,0,0,37,10,1
1,3,0,0,6,0,42,91,0,32,0,...,0,0,0,0,0,0,2,2838,46,1
2,2,0,0,4,0,23,3,0,17,2,...,0,0,0,0,0,0,1,111,20,1
3,1,0,0,4,0,27,9,0,36,0,...,0,0,0,0,0,0,7,987,197,1
4,3,0,0,11,0,18,3,0,16,0,...,0,0,0,0,0,0,1,98,25,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11593,2,0,0,11,0,61,13,0,66,17,...,0,0,0,0,0,0,10,2764,236,5
11594,6,0,0,10,1,69,50,1,118,15,...,0,0,0,0,1,0,5,4417,295,5
11595,0,0,0,0,0,21,16,0,48,9,...,0,0,0,0,0,0,5,241,67,5
11596,1,0,0,15,0,90,80,0,239,50,...,0,0,0,0,0,0,3,1703,774,5


In [33]:
df=pd.read_csv(r'E:\project sarmin mam\CSV\feature_vectors_static.csv')
df

KeyboardInterrupt: 

In [63]:
df=pd.read_csv(r"E:\project sarmin mam\CSV\feature_vectors_syscalls_frequency_5_Cat.csv")
df

,__arm_nr_cacheflush,__arm_nr_set_tls,_llseek,_newselect,accept,access,bind,brk,capset,chdir,...,ugetrlimit,umask,uname,unlink,utimes,vfork,wait4,write,writev,Class
0,0.0,14.0,6.0,0.0,0.0,11.0,3.0,21.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,37.0,10.0,1.0
1,1590.0,42.0,6.0,0.0,0.0,47.0,2.0,1430.0,1.0,0.0,...,2.0,0.0,1.0,4.0,0.0,0.0,0.0,2840.0,46.0,1.0
2,0.0,23.0,6.0,0.0,0.0,40.0,9.0,89.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,111.0,20.0,1.0
3,0.0,27.0,6.0,0.0,0.0,60.0,1.0,242.0,1.0,0.0,...,0.0,0.0,1.0,2.0,0.0,0.0,0.0,987.0,197.0,1.0
4,0.0,18.0,6.0,0.0,0.0,26.0,5.0,59.0,1.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,98.0,25.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11593,3120.0,61.0,12.0,34.0,0.0,772.0,12.0,667.0,1.0,0.0,...,3.0,0.0,2.0,21.0,0.0,0.0,0.0,2760.0,236.0,5.0
11594,39000.0,70.0,66.0,20.0,0.0,147.0,8.0,995.0,1.0,0.0,...,3.0,0.0,1.0,21.0,0.0,0.0,1.0,4420.0,295.0,5.0
11595,0.0,21.0,112.0,9.0,0.0,97.0,1.0,194.0,1.0,0.0,...,0.0,0.0,1.0,9.0,0.0,0.0,0.0,241.0,67.0,5.0
11596,0.0,90.0,115.0,75.0,0.0,525.0,23.0,349.0,1.0,0.0,...,1.0,0.0,1.0,56.0,0.0,0.0,0.0,1700.0,774.0,5.0


In [ ]:
df.isna().sum()

In [ ]:
len(df.columns)

In [65]:
df = df.apply(lambda x: x.astype(float).astype(int))

In [72]:
d=df.iloc[:,-10:-1]
d

,ugetrlimit,umask,uname,unlink,utimes,vfork,wait4,write,writev
0,0,0,1,0,0,0,0,37,10
1,2,0,1,4,0,0,0,2840,46
2,0,0,1,2,0,0,0,111,20
3,0,0,1,2,0,0,0,987,197
4,0,0,1,0,0,0,0,98,25
...,...,...,...,...,...,...,...,...,...
11593,3,0,2,21,0,0,0,2760,236
11594,3,0,1,21,0,0,1,4420,295
11595,0,0,1,9,0,0,0,241,67
11596,1,0,1,56,0,0,0,1700,774


In [70]:
d.columns

Index(['ugetrlimit', 'umask', 'uname', 'unlink', 'utimes', 'vfork', 'wait4',
       'write', 'writev'],
      dtype='object')

In [54]:
pip install apk_parse

     ---------------------------------------- 50.2/50.2 kB 2.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from apk_parse.apk import APK

def read_apk(apk_path):
    apk = APK(apk_path)
    print(f"Package: {apk.package}")
    print(f"Version Code: {apk.version_code}")
    print(f"Version Name: {apk.version_name}")
    print(f"App Name: {apk.application}")

# Replace 'path_to_your_apk_file.apk' with the actual file path



In [55]:
read_apk("C:\Users\Lenovo\Downloads\Adware-APKs\Adware\feiwo\b9b5ac2c9ea9c72c220d560bcaf265b7.apk")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (1916588364.py, line 1)

In [60]:
import pandas as pd

# Sample DataFrame
data = {'Name': ['Alice', 'Bob', 'Charlie'], 'Age': [25, 30, 35], 'City': ['New York', 'Los Angeles', 'Chicago']}
df = pd.DataFrame(data)
df

# Accessing a column by its number (e.g., the second column)



,Name,Age,City
0,Alice,25,New York
1,Bob,30,Los Angeles
2,Charlie,35,Chicago


In [62]:
 df.iloc[:, column_number]

0    25
1    30
2    35
Name: Age, dtype: int64